In [1]:
print("Input dataset name:")
dataset = raw_input()
#dataset = 'HTo2LongLivedTo4b_MH-125_MFF-25_CTau-15000mm_TuneCP5_14TeV_pythia8'

Input dataset name:
HTo2LongLivedTo4q_MH_125_MFF_1_CTau_5000mm_TuneCP5_14TeV_pythia


In [2]:
from ROOT import TFile, TTree
from ROOT import gROOT, AddressOf
import uproot
from array import array
from tqdm import tqdm
import numpy as np

Welcome to JupyROOT 6.14/09


In [3]:
comp_ME11 = array('f', [0.])
comp_ME12 = array('f', [0.])
comp_ME13 = array('f', [0.])
comp_ME21 = array('f', [0.])
comp_ME22 = array('f', [0.])
comp_ME31 = array('f', [0.])
comp_ME32 = array('f', [0.])
comp_ME41 = array('f', [0.])
comp_ME42 = array('f', [0.])

wire_ME11 = array('f', [0.])
wire_ME12 = array('f', [0.])
wire_ME13 = array('f', [0.])
wire_ME21 = array('f', [0.])
wire_ME22 = array('f', [0.])
wire_ME31 = array('f', [0.])
wire_ME32 = array('f', [0.])
wire_ME41 = array('f', [0.])
wire_ME42 = array('f', [0.])


In [4]:
in_path = "/uscms/home/menendez/nobackup/Trigger/CMSSW_10_6_4/src/Data/TPEHists_" + dataset + ".root"
out_path = "trees/MaxDigis_tree_" + dataset + ".root"

f = TFile( out_path, 'RECREATE' )
comp_tree = TTree( 'comparator', 'Max Comparator digis in a single chamber in each ring per event' )
wire_tree = TTree( 'wire', 'Max Comparator digis in a single chamber in each ring per event' )
# tree.Branch( 'digis', digis, 'Ev_max_nComp_ME11/I:Ev_max_nComp_ME12:Ev_max_nComp_ME13:Ev_max_nComp_ME21:Ev_max_nComp_ME22:Ev_max_nComp_ME31:Ev_max_nComp_ME32:Ev_max_nComp_ME41:Ev_max_nComp_ME42:Ev_max_nWire_ME11:Ev_max_nWire_ME12:Ev_max_nWire_ME13:Ev_max_nWire_ME21:Ev_max_nWire_ME22:Ev_max_nWire_ME31:Ev_max_nWire_ME32:Ev_max_nWire_ME41:Ev_max_nWire_ME42',0 )
comp_tree.Branch( 'Ev_max_nComp_ME11', comp_ME11, 'comp_ME11/F')
comp_tree.Branch( 'Ev_max_nComp_ME12', comp_ME12, 'comp_ME12/F')
comp_tree.Branch( 'Ev_max_nComp_ME13', comp_ME13, 'comp_ME13/F')
comp_tree.Branch( 'Ev_max_nComp_ME21', comp_ME21, 'comp_ME21/F')
comp_tree.Branch( 'Ev_max_nComp_ME22', comp_ME22, 'comp_ME22/F')
comp_tree.Branch( 'Ev_max_nComp_ME31', comp_ME31, 'comp_ME31/F')
comp_tree.Branch( 'Ev_max_nComp_ME32', comp_ME32, 'comp_ME32/F')
comp_tree.Branch( 'Ev_max_nComp_ME41', comp_ME41, 'comp_ME41/F')
comp_tree.Branch( 'Ev_max_nComp_ME42', comp_ME42, 'comp_ME42/F')

wire_tree.Branch( 'Ev_max_nWire_ME11', wire_ME11, 'wire_ME11/F')
wire_tree.Branch( 'Ev_max_nWire_ME12', wire_ME12, 'wire_ME12/F')
wire_tree.Branch( 'Ev_max_nWire_ME13', wire_ME13, 'wire_ME13/F')
wire_tree.Branch( 'Ev_max_nWire_ME21', wire_ME21, 'wire_ME21/F')
wire_tree.Branch( 'Ev_max_nWire_ME22', wire_ME22, 'wire_ME22/F')
wire_tree.Branch( 'Ev_max_nWire_ME31', wire_ME31, 'wire_ME31/F')
wire_tree.Branch( 'Ev_max_nWire_ME32', wire_ME32, 'wire_ME32/F')
wire_tree.Branch( 'Ev_max_nWire_ME41', wire_ME41, 'wire_ME41/F')
wire_tree.Branch( 'Ev_max_nWire_ME42', wire_ME42, 'wire_ME42/F')



<ROOT.TBranch object ("Ev_max_nWire_ME42") at 0x5b36f30>

In [5]:
if dataset=="Data":
  MC = False
  Neutrino = False
elif dataset=="Neutrino_gun" or dataset=="Nu_E10-pythia8-gun":
  MC = False
  Neutrino = True
else:
  MC = True
  Neutrino = False

if MC:
  File = TFile(in_path,"READ")
elif Neutrino:
  File = TFile("/uscms/home/menendez/nobackup/Trigger/CMSSW_10_6_4/src/Data/TPEHists_Neutrino_gun.root","READ")
else:
  File = TFile("/uscms/home/menendez/nobackup/Trigger/CMSSW_10_6_4/src/Data/TPEHists_Data.root","READ")
clct = File.Get("lctreader/Ev_clcttree")
alct = File.Get("lctreader/Ev_alcttree")
llp = File.Get("lctreader/llp")

nEntries_clct = clct.GetEntries()
nEntries_alct = alct.GetEntries()
if MC:
  nEntries_llp = llp.GetEntries()

In [6]:
llp_accept = []
#max_nComp = np.array([0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0])
max_nComp = np.zeros((5,4))
max_nWire = np.zeros((5,4))
lastEventclct = 0
lastEventalct = 0

In [7]:
if MC:
  print "Finding LLP Acceptance"
  for i in tqdm(range(0, nEntries_llp)):
    llp.GetEntry(i)

    if(llp.llp_in_acceptance[0]==1 or llp.llp_in_acceptance[1]==1):
      llp_accept.append(llp.event)

  0%|          | 0/48500 [00:00<?, ?it/s]

Finding LLP Acceptance


100%|██████████| 48500/48500 [00:03<00:00, 13210.75it/s]


In [8]:
print "Starting CLCT Analysis"
for i in tqdm(range(0, nEntries_clct)):
  clct.GetEntry(i)

  if MC:
    if clct.t_Event not in llp_accept:
      continue
    
  if(clct.t_Event!=lastEventclct):
    lastEventclct = clct.t_Event
    
    comp_ME11[0] = max_nComp[1][1]
    comp_ME12[0] = max_nComp[1][2]
    comp_ME13[0] = max_nComp[1][3]
    comp_ME21[0] = max_nComp[2][1]
    comp_ME22[0] = max_nComp[2][2]
    comp_ME31[0] = max_nComp[3][1]
    comp_ME32[0] = max_nComp[3][2]
    comp_ME41[0] = max_nComp[4][1]
    comp_ME42[0] = max_nComp[4][2]
    max_nComp = np.zeros((5,4))

    comp_tree.Fill()
       
        
  if clct.t_nComp > max_nComp[clct.t_station][clct.t_ring]:
    max_nComp[clct.t_station][clct.t_ring] = clct.t_nComp
                    

 16%|█▋        | 6430/39165 [00:00<00:00, 64299.29it/s]

Starting CLCT Analysis


100%|██████████| 39165/39165 [00:00<00:00, 73389.70it/s]


In [9]:
print "Starting ALCT Analysis"
for i in tqdm(range(0, nEntries_alct)):
  alct.GetEntry(i)

  if MC:
    if alct.t_Event not in llp_accept:
      continue
    
  if(alct.t_Event!=lastEventalct):
    lastEventalct = alct.t_Event
    
    wire_ME11[0] = max_nWire[1][1]
    wire_ME12[0] = max_nWire[1][2]
    wire_ME13[0] = max_nWire[1][3]
    wire_ME21[0] = max_nWire[2][1]
    wire_ME22[0] = max_nWire[2][2]
    wire_ME31[0] = max_nWire[3][1]
    wire_ME32[0] = max_nWire[3][2]
    wire_ME41[0] = max_nWire[4][1]
    wire_ME42[0] = max_nWire[4][2]
    max_nWire = np.zeros((5,4))

    wire_tree.Fill()
  

  if alct.t_nWire > max_nWire[alct.t_station][alct.t_ring]:
    max_nWire[alct.t_station][alct.t_ring] = alct.t_nWire
    

 16%|█▌        | 6591/41828 [00:00<00:00, 65905.98it/s]

Starting ALCT Analysis


100%|██████████| 41828/41828 [00:00<00:00, 68932.09it/s]


In [10]:
f.Write()
f.Close()